In [2]:
import sys
sys.path.append("/Users/macknixon/Projects/arc/ner-ocr")

In [2]:
from src.bounding_box import PaddleOCRWrapper

/Users/macknixon/Projects/arc/ner-ocr/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
extractor = PaddleOCRWrapper(ocr_timeout=500)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.p

In [8]:
res = extractor.extract("../data/input/vol_fil.pdf")

2025-12-18 16:07:11.289 | INFO     | src.bounding_box:extract:273 - Processing file: ../data/input/vol_fil.pdf (extension: .pdf)
2025-12-18 16:07:11.290 | INFO     | src.pdf_processing:pdf_to_images:39 - Converting PDF to images: ../data/input/vol_fil.pdf
2025-12-18 16:07:11.294 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 0
2025-12-18 16:07:11.360 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 1
2025-12-18 16:07:11.410 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 2
2025-12-18 16:07:11.465 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 3
2025-12-18 16:07:11.517 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 4
2025-12-18 16:07:11.567 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 5
2025-12-18 16:07:11.617 | DEBUG    | src.pdf_processing:pdf_to_images:54 - Processing PDF page 6
2025-12-18 16:07:11.664 | DEBUG    | src.pdf_processing:pdf_to_im

In [ ]:
from paddleocr import PaddleOCR

ocr = PaddleOCR()
result = ocr.predict("../data/input/vol_fil.pdf")

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/macknixon/.p

In [23]:
for i, res in enumerate(result):
    res.save_to_img(f"../data/output/vol_fil_ocr_output_{i}.png")

The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_0.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_1.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_2.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_3.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_4.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_5.png`!
The result has multiple img files need to be saved. But the `save_path` has been specified as `../data/output/vol_fil_ocr_output_6.png`!
The result has multiple img files need to

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoProcessor, AutoTokenizer
from qwen_vl_utils import process_vision_info
from dots_ocr.utils import dict_promptmode_to_prompt

model_path = "../DotsOCR/weights/DotsOCR"
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)

image_path = "../data/input/pms_annotated-1.png"
prompt = """Please output the layout information from the PDF image, including each layout element's bbox, its category, and the corresponding text content within the bbox.

1. Bbox format: [x1, y1, x2, y2]

2. Layout Categories: The possible categories are ['Caption', 'Footnote', 'Formula', 'List-item', 'Page-footer', 'Page-header', 'Picture', 'Section-header', 'Table', 'Text', 'Title'].

3. Text Extraction & Formatting Rules:
    - Picture: For the 'Picture' category, the text field should be omitted.
    - Formula: Format its text as LaTeX.
    - Table: Format its text as HTML.
    - All Others (Text, Title, etc.): Format their text as Markdown.

4. Constraints:
    - The output text must be the original text from the image, with no translation.
    - All layout elements must be sorted according to human reading order.

5. Final Output: The entire output must be a single JSON object.
"""

messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path
                },
                {"type": "text", "text": prompt}
            ]
        }
    ]

# Preparation for inference
text = processor.apply_chat_template(
    messages, 
    tokenize=False, 
    add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=24000)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

/Users/macknixon/Projects/arc/ner-ocr/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not available! fallback to eager implementation 
flash attention not avail

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.25s/it]
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


TypeError: Received a NoneType for argument video_processor, but a BaseVideoProcessor was expected.